# Gerenciador de finanças

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
from plotly import tools
import plotly.graph_objs as go
# from lib import lib as l
from lib import plot as p
from lib import data as d
from lib import finance as f

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
lpalette = {'g':'#13E881', 'r':'#FF5949', 'o':'#FFB84C', 'b':'#A7BEFA', 'w':'#FFFFFF', 'pk':'#FFA1FD', 'bg':'#FFEDBA', 'c':'#90EBF0',
            'pp':'#BBA0F0', 'g2':'#CCFFED', 'b2':'#E3EEFF', 'y':'#FFFC57'}
dpalette = {'g':'#268040', 'r':'#800B00', 'o':'#A13808', 'b':'#464CC2', 'w':'#B8BFBA', 'pk':'#A04EA6', 'bg':'#807155', 'c':'#1D4544',
            'pp':'#291147', 'g2':'#394742', 'b2':'#414449', 'y':'#666523'}

#### Leitura dos dados

In [3]:
dg, df, dtr = f.read_finance()

## Saldo atual

In [4]:
f.calc_account_funds(dg, df, dtr, [603.08, 0, 514.75, 7975.6,24344.27])

,Saldo,Lido,Erro
,,,
Alelo,603.080,603.080,-0.000
BB,0.000,0.000,0.000
Cash,514.750,514.750,0.000
Easy,7975.600,7975.600,-0.000
Nu,24344.270,24344.270,-0.000


## Análise mensal

In [5]:
monthly_income = d.window(d.pivot_agg(dg, val='Valor', idx=['Data'], agg={'Valor':np.sum}))
monthly_expenses = d.window(d.pivot_agg(df, val='Valor', idx=['Data'], agg={'Valor':np.sum}))
monthly_expenses_p = d.window(d.pivot_agg(df[df['Pagamento'] != 'Pais'], val='Valor', idx=['Data'], agg={'Valor':np.sum}))

monthly_income.columns = ['Data', 'Renda', 'Renda média']
monthly_expenses.columns = ['Data', 'Despesa', 'Despesa média']
monthly_expenses_p.columns = ['Data', 'Despesa real', 'Despesa real média']

balance = monthly_income.join(monthly_expenses.set_index('Data'), on='Data').join(monthly_expenses_p.set_index('Data'), on='Data')

### Balanço

In [6]:
fig_bal = p.make_fig()

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [7]:
palette = {'Renda média': dpalette['g'], 'Despesa média': dpalette['r'], 'Despesa real média': dpalette['o'],
           'Renda': lpalette['g'], 'Despesa': lpalette['r'], 'Despesa real': lpalette['o']}
p.plot(balance, 'Data', fig_bal, palette, ['Renda média', 'Despesa média', 'Despesa real média'])
p.plot(balance, 'Data', fig_bal, palette, ['Renda', 'Despesa', 'Despesa real'], 'bar')

FigureWidget({
    'data': [{'hoverinfo': 'name+y',
              'marker': {'color': '#268040'},
            …

### Saldo

In [8]:
fund = d.window(f.calc_fund(balance.fillna(0)), val='Var')
fund_p = d.window(f.calc_fund(balance.fillna(0), expense='Despesa'), val='Var')

In [9]:
fig_fund = p.make_fig(cols=2, specs=[[{}, {}]])
fig_grow = p.make_fig()
fig_var = p.make_fig()

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [10]:
p.plot(fund, 'Data', fig_fund, {'Saldo': lpalette['b']}, y=['Saldo'], plot_type='area', name=['Saldo real'])
p.plot(fund_p, 'Data', fig_fund, {'Saldo': lpalette['r']}, y=['Saldo'], plot_type='area', col=2)

FigureWidget({
    'data': [{'hoverinfo': 'name+y',
              'marker': {'color': '#A7BEFA'},
            …

In [11]:
p.plot(fund, 'Data', fig_grow, {'%': lpalette['b']}, y=['%'], name=['Crescimento real'])
p.plot(fund_p, 'Data', fig_grow, {'%': lpalette['r']}, y=['%'], name=['Crescimento'])

FigureWidget({
    'data': [{'hoverinfo': 'name+y',
              'marker': {'color': '#A7BEFA'},
            …

In [12]:
p.plot(fund, 'Data', fig_var, {'Var': lpalette['b']}, y=['Var'], plot_type='bar', name=['Variação real'])
p.plot(fund, 'Data', fig_var, {'VarAvg': dpalette['b']}, y=['VarAvg'], name=['Variação real média'])
p.plot(fund_p, 'Data', fig_var, {'Var': lpalette['r']}, y=['Var'], plot_type='bar', name=['Variação'])
p.plot(fund_p, 'Data', fig_var, {'VarAvg': dpalette['r']}, y=['VarAvg'], name=['Variação média'])

FigureWidget({
    'data': [{'hoverinfo': 'name+y',
              'marker': {'color': '#A7BEFA'},
            …

## Matriz de finanças

In [13]:
de = df[df['Area'] != 'Credito']

area_income = d.pivot_agg(dg, val='Valor', idx=['Data'], col=['Area'], agg={'Valor':np.sum})
area_income_pct = d.row_percentage(area_income.set_index('Data')).reset_index()

area_expense = d.pivot_agg(de[de['Pagamento'] != 'Pais'], val='Valor', idx=['Data'], col=['Area'], agg={'Valor':np.sum})
area_expense_pct = d.row_percentage(area_expense.set_index('Data')).reset_index()

dtrip = de[de['Area'] == 'Viagem']
trip_expense = d.pivot_agg(dtrip, val='Valor', idx=['Data'], col=['Sub'], agg={'Valor':np.sum})
trip_expense_pct = d.row_percentage(trip_expense.set_index('Data')).reset_index()

# dtrip = de[de['Area'] == 'Viagem']
# trip_expense = d.pivot_agg(dtrip, val='Valor', idx=['Data'], col=['Sub'], agg={'Valor':np.sum})
# trip_expense_pct = d.row_percentage(area_expense.set_index('Data')).reset_index()

area_expense_p = d.pivot_agg(de, val='Valor', idx=['Data'], col=['Area'], agg={'Valor':np.sum})
area_expense_p_pct = d.row_percentage(area_expense_p.set_index('Data')).reset_index()

### Gastos

In [14]:
fig_exp = p.make_fig()
fig_exp_pct = p.make_fig(cols=3, specs=[[{'colspan':2},{},{}]])
fig_exp_p = p.make_fig()
fig_exp_p_pct = p.make_fig(cols=3, specs=[[{'colspan':2},{},{}]])
fig_trip = p.make_fig()
fig_trip_pct = p.make_fig(cols=3, specs=[[{'colspan':2},{},{}]])

This is the format of your plot grid:
[ (1,1) x1,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1    [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1    [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1    [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]



### Real

In [15]:
palette = {'Alimentacao': lpalette['g'], 'Viagem': lpalette['c'], 'Eletronicos': dpalette['b'], 'Presente': lpalette['pk'], 'Imposto': lpalette['r'],
           'Estudo': dpalette['g2'], 'Transporte': lpalette['o'], 'Banco': dpalette['pp'], 'Moradia':lpalette['y'], 'Passeio':dpalette['b2']}
p.plot(area_expense, 'Data', fig_exp, palette, plot_type='area')

FigureWidget({
    'data': [{'hoverinfo': 'name+y',
              'marker': {'color': '#13E881'},
            …

In [16]:
fig_exp_pct = p.plot(area_expense_pct, 'Data', fig=fig_exp_pct, row=1, col=1, palette=palette, plot_type='area')
temp_sum = area_expense.sum()
area_expense_sum = temp_sum.drop(temp_sum.index[0])
p.plot_pie(area_expense_sum, fig_exp_pct, palette, dict(x=[0.68, 1]))

FigureWidget({
    'data': [{'hoverinfo': 'name+y',
              'marker': {'color': '#13E881'},
            …

### Total

In [17]:
p.plot(area_expense_p, 'Data', fig_exp_p, palette, plot_type='area')

FigureWidget({
    'data': [{'hoverinfo': 'name+y',
              'marker': {'color': '#13E881'},
            …

In [18]:
fig_exp_p_pct = p.plot(area_expense_p_pct, 'Data', fig=fig_exp_p_pct, row=1, col=1, palette=palette, plot_type='area')
temp_sum = area_expense_p.sum()
area_expense_p_sum = temp_sum.drop(temp_sum.index[0])
p.plot_pie(area_expense_p_sum, fig_exp_p_pct, palette, dict(x=[0.68, 1]))

FigureWidget({
    'data': [{'hoverinfo': 'name+y',
              'marker': {'color': '#13E881'},
            …

### Trip

In [19]:
palette = {'Alimentacao': lpalette['g'], 'Voo': lpalette['c'], 'Hotel': dpalette['b'], 'Farma': lpalette['pk'], 'Snack': lpalette['r'],
           'Passeio': dpalette['g2'], 'Presente': lpalette['o'], 'Mercado': dpalette['pp'], 'Transporte':lpalette['y']}
p.plot(trip_expense, 'Data', fig_trip, palette, plot_type='area')

FigureWidget({
    'data': [{'hoverinfo': 'name+y',
              'marker': {'color': '#13E881'},
            …

In [20]:
fig_trip_pct = p.plot(trip_expense_pct, 'Data', fig=fig_trip_pct, row=1, col=1, palette=palette, plot_type='area')
temp_sum = trip_expense.sum()
trip_expense_sum = temp_sum.drop(temp_sum.index[0])
p.plot_pie(trip_expense_sum, fig_trip_pct, palette, dict(x=[0.68, 1]))

FigureWidget({
    'data': [{'hoverinfo': 'name+y',
              'marker': {'color': '#13E881'},
            …

### Ganhos

In [21]:
fig_inc = p.make_fig()
fig_inc_pct = p.make_fig(cols=3, specs=[[{'colspan':2},{},{}]])

This is the format of your plot grid:
[ (1,1) x1,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1    [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]



In [22]:
palette = {'Pais': lpalette['r'], 'Investimento': lpalette['y'], 'Salario': lpalette['g']}
p.plot(area_income, 'Data', fig_inc, palette, plot_type='area')

FigureWidget({
    'data': [{'hoverinfo': 'name+y',
              'marker': {'color': '#FFFC57'},
            …

In [23]:
fig_inc_pct = p.plot(area_income_pct, 'Data', fig=fig_inc_pct, row=1, col=1, palette=palette, plot_type='area')
temp_sum = area_income.sum()
area_income_sum = temp_sum.drop(temp_sum.index[0])
p.plot_pie(area_income_sum, fig_inc_pct, palette, dict(x=[0.68, 1]))

FigureWidget({
    'data': [{'hoverinfo': 'name+y',
              'marker': {'color': '#FFFC57'},
            …